In [0]:
!wget -O backdoor.zip http://jacarini.dinf.usherbrooke.ca/static/dataset/shadow/backdoor.zip
!wget -O pedestrian.zip http://jacarini.dinf.usherbrooke.ca/static/dataset/baseline/pedestrians.zip
!wget -O cubicle.zip http://jacarini.dinf.usherbrooke.ca/static/dataset/shadow/cubicle.zip
!wget -O busStation.zip http://jacarini.dinf.usherbrooke.ca/static/dataset/shadow/busStation.zip
!wget -O peopleinshadow.zip http://jacarini.dinf.usherbrooke.ca/static/dataset/shadow/peopleInShade.zip

!unzip backdoor.zip
!unzip pedestrian.zip
!unzip cubicle.zip
!unzip busStation.zip
!unzip peopleinshadow.zip

!rm backdoor.zip
!rm pedestrian.zip
!rm cubicle.zip
!rm busStation.zip
!rm peopleinshadow.zip

!mkdir ./detections/
!wget -O ./detections/detections.zip http://jacarini.dinf.usherbrooke.ca/static/pedestrian%20detection//pedestrian%20detection%20dataset.zip
!unzip ./detections/detections.zip -d ./detections/
!rm ./detections/detections.zip

In [0]:
files = {}
aa = lambda x, y: list(range(x, y))
files["backdoor"] = {"background": [(400, 1358), (1561, 1629), (1995, 2000)],
                     "foreground": [aa(1404, 1524), 
                                    aa(1658, 1742), 
                                    aa(1833, 1870) + aa(1897, 1929)]}
files["cubicle"] = {"background": [(1416, 1557), (1817, 1951), (6641, 6924), (7181, 7400)],
                    "foreground": [aa(1172, 1398),
                                   aa(1611, 1801),
                                   aa(1972, 2174),
                                   aa(2308, 2522),
                                   aa(6035, 6152),
                                   aa(6323, 6453),
                                   aa(6500, 6571),
                                   aa(6601, 6629),
                                   aa(6934, 7121)]}
files["pedestrians"] = {"background": [(713, 826)], 
                        "foreground": [aa(318, 401),
                                       aa(511, 556), 
                                       aa(602, 694), 
                                       aa(854, 1035), 
                                       aa(1056, 1099)]}
files["peopleInShade"] = {"background":[(250, 284), (371, 434), (509, 573), 
                                         (808, 825), (989, 1056)], 
                           "foreground":[aa(292, 313), 
                                         aa(451, 502), 
                                         aa(589, 796), 
                                         aa(845, 858)]}

In [0]:
import os 
import numpy as np
import cv2 as cv
import random

savepath = "./dataset/"
readpath = "./detections/pedestrian detection dataset/"
if not os.path.isdir(savepath):
    os.makedirs(savepath)
if not os.path.isdir("./dataset/gt/"):
    os.makedirs("./dataset/gt/")
if not os.path.isdir("./dataset/in/"):
    os.makedirs("./dataset/in/")
if not os.path.isdir("./dataset/bk/"):
    os.makedirs("./dataset/bk/")

In [0]:
def syn(persons, gts, back, overlap, seed):
    finalgt = np.zeros_like(gts[0])
    finalim = back.copy()
    orders = list(range(len(persons)))
    random.seed(seed)
    random.shuffle(orders)
    xywh = []
    seedi = seed + 119
    seedj = seed + 2843

    for i, order in enumerate(orders):
        m = 0
        foreground = persons[order]
        groundtruth = gts[order]
            
        pixels = np.where(groundtruth[:,:,0] >160)
        ymin = np.min(pixels[0])
        ymax = np.max(pixels[0])
        xmin = np.min(pixels[1])
        xmax = np.max(pixels[1])
        
        newgt = groundtruth[ymin:ymax, xmin:xmax,:]
        newgt[newgt < 200] = 0
        newgt[newgt > 200] = 1
        newim = foreground[ymin:ymax, xmin:xmax,:]
        
        random.seed(seed + 3)
        newsize = random.gauss(0.9, 0.1)
        newsize = 1 if newsize > 1 else newsize
        newsize = 0.7 if newsize < 0.7 else newsize
        newim = cv.resize(newim, None, fx=newsize, fy=newsize)
        newgt = cv.resize(newgt, None, fx=newsize, fy=newsize)

        random.seed(seed + 175)
        if random.uniform(0, 1)>0.5:
            newim = newim[:, ::-1, :]
            newgt = newgt[:, ::-1, :]
        
        if overlap and i==1:
            while True:
                seedi += 1
                m += 1
                n = 0
                if m > 1000:
                    return finalim, finalgt
                x0, y0, w0, h0 = xywh[0]
                top = y0 - h0 * 3 // 2
                left = x0 - w0 * 3 // 2
                right = x0 + w0 * 3 // 2
                bottom = y0 + h0 * 3 // 2
                top = 0 if top < 0 else top
                left = 0 if left < 0 else left
                right = back.shape[1] if right > back.shape[1] else right
                bottom = back.shape[0] if bottom > back.shape[0] else bottom
                while True:
                    n += 1
                    if n > 2000:
                        return finalim, finalgt
                    seedj += 98
                    random.seed(seedi + seedj)
                    y= random.randint(top, bottom)
                    random.seed((seedi + seedj) //32 )
                    x = random.randint(left, right)

                    if y+newim.shape[0] < back.shape[0] and x+newim.shape[1] < back.shape[1]:
                        break
                gtt = np.zeros_like(groundtruth)
                gtt[y:y+newim.shape[0], x:x+newgt.shape[1]] = newgt
                n2and1 = len(np.where((gtt * finalgt) > 0)[0])
                n1 = len(np.where(gtt > 0)[0])
                n2 = len(np.where(gtt > 0)[0])
                if n2and1 < min(n1, n2) * 0.7 and n2and1 > min(n1, n2) * 0.2:
                    break
        else:
            while True:
                seedi += 1
                m += 1
                n = 0
                if m > 1000:
                    return finalim, finalgt
                while True:
                    seedj += 1
                    n += 1
                    if n > 2000:
                        return finalim, finalgt
                    random.seed(seedi + seedj + 21)
                    y = random.randint(0, back.shape[0])
                    random.seed(seedi + seedj + 981)
                    x = random.randint(0, back.shape[1])
                    if y+newim.shape[0] < back.shape[0] and x+newim.shape[1] < back.shape[1]:
                        break
                gtt = np.zeros_like(groundtruth)
                gtt[y:y+newim.shape[0], x:x+newgt.shape[1]] = newgt
                n2and1 = len(np.where((gtt * finalgt) > 0)[0])
                n1 = len(np.where(gtt > 0)[0])
                n2 = len(np.where(gtt > 0)[0])
                if n2and1 < min(n1, n2) * 0.5:
                    break
        xywh.append((x, y, newgt.shape[1], newgt.shape[0]))
        imm = np.zeros_like(groundtruth)
        imm[y:y+newim.shape[0], x:x+newim.shape[1]] = newim

        finalgt[gtt!=0] = 1 + order
        finalim = finalim * (1-gtt) + imm * (gtt)
        
    return finalim, finalgt

In [0]:
def loadimage(path, mode, i):
    middle = "/groundtruth/" if mode == "gt" else "/input/"
    prefix = ".png" if mode == "gt" else ".jpg"
    filename = path + middle + mode + str(i).zfill(6) + prefix
    # print(filename)
    im = cv.imread(filename)
    im = cv.cvtColor(im, cv.COLOR_BGR2RGB)
    return im

In [0]:
import torch 

n_frames = 3000
max_person = 3

targets = [None] * n_frames

for i in range(n_frames):
    if i % 100 == 0:
        print(i)
    random.seed(i+1201)
    n_person = random.randint(2, max_person)
    random.seed(i*72)
    overlap = False if random.uniform(0, 1) < 0.7 else True
    overlap = False if n_person < 2 else overlap
    random.seed(i)
    f = i // (n_frames // 4)
    f = list(files.keys())[f]
    background = files[f]["background"]
    random.seed(i + 192)
    background = background[random.randint(0, len(background)-1)]
    random.seed(i + 212)
    background = random.randint(background[0], background[1] - 1)
    background = loadimage(f, 'in', background)
    foreground = files[f]["foreground"]

    used = []
    prs = []
    gts = []
    for j in range(0, n_person):
        seedj = 0
        while True:
            seedj += 1
            random.seed(seedj + 1982)
            foreGroup = random.randint(0, len(foreground) - 1)
            if foreGroup not in used:
                break
        used.append(foreGroup)
        random.seed(i + 2763)
        fore = random.randint(0, len(foreground[foreGroup]) - 1)
        fore = foreground[foreGroup][fore]
        person = loadimage(f, 'in', fore)
        ground = loadimage(f, 'gt', fore)
        ground[ground<160] = 0
        retval, labels = cv.connectedComponents(ground[:,:,0])
        
        for k in range(0, retval):
            im = np.zeros((labels.shape[0], labels.shape[1], 3))
            mask = np.zeros_like(ground)
            mask[ground>200] = 1
            im[labels == k, :] = 1
            im[labels != k, :] = 0
            im[im>0] = ground[im>0]
            im = im * mask
            if np.max(im) > 0:
                prs.append(person)
                gts.append(im)
        
    a = aa(0, len(prs))
    random.seed(i)
    random.shuffle(a)
    a = a[:n_person]
    prs = [prs[i] for i in a]
    gts = [gts[i] for i in a]
    
    f, g = syn(prs, gts, background, overlap, i)
    f = cv.cvtColor(np.uint8(f), cv.COLOR_RGB2BGR)
    boxes = []
    obj_ids = np.unique(g)[1:]
    for idx in obj_ids:
        pos = np.where(g==idx)
        xmin = np.min(pos[1])
        xmax = np.max(pos[1])
        ymin = np.min(pos[0])
        ymax = np.max(pos[0])
        boxes.append([xmin, ymin, xmax, ymax])
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    labels = torch.ones((len(boxes),), dtype=torch.int64)
    image_id = torch.tensor([i])
    area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
    iscrowd = torch.zeros((len(boxes),), dtype=torch.int64)
    target = {}
    target["boxes"] = boxes
    target["labels"] = labels
    target["image_id"] = image_id
    target["area"] = area
    target["iscrowd"] = iscrowd
    targets[i] = target

    cv.imwrite("/content/dataset/gt/" + str(i).zfill(6) + '.png', g)
    cv.imwrite("/content/dataset/in/" + str(i).zfill(6) + '.jpg', f)
    cv.imwrite("/content/dataset/bk/" + str(i).zfill(6) + '.jpg', background)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900


In [0]:
random.seed(19872)
idx = list(range(0, n_frames))
random.shuffle(idx)
trainset = idx[:n_frames//6*4]
valset = idx[n_frames//6*4:n_frames//6*5]
testset = idx[n_frames//6*5:]

In [0]:
from PIL import Image
import torch
import torchvision
import torchvision.transforms as T
import numpy as np

from torchvision.models.detection.transform import GeneralizedRCNNTransform
from torchvision.models.detection.generalized_rcnn import GeneralizedRCNN

from collections import OrderedDict
from torch import nn
import warnings
from torch.jit.annotations import Tuple, List, Dict, Optional
from torch import Tensor
from copy import deepcopy

In [0]:
STEPS = 32
class NETC(GeneralizedRCNN):
    def __init__(self, backbone, rpn, roi_heads, transform):
        super().__init__(backbone=backbone, 
                         rpn=rpn, 
                         roi_heads=roi_heads, 
                         transform=transform)
        self.MergeNet = nn.Sequential(nn.Conv2d(2*STEPS, 2*STEPS, 1, bias=False),
                                      nn.BatchNorm2d(2*STEPS, affine=False),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(2*STEPS, STEPS, 1, bias=False),
                                      nn.BatchNorm2d(STEPS, affine=False),
                                      nn.ReLU(inplace=True))
    
    def forward(self, xs, ys, targets=None):
        if self.training and targets is None:
            raise ValueError("In training mode, targets should be passed")
        original_image_sizes = torch.jit.annotate(List[Tuple[int, int]], [])
        for img in xs:
            val = img.shape[-2:]
            assert len(val) == 2
            original_image_sizes.append((val[0], val[1]))

        xs, targets = self.transform(xs, targets)
        ys, _ = self.transform(ys, None)
        
        features1 = self.backbone(xs.tensors)
        features2 = self.backbone(ys.tensors)
        features3 = deepcopy(features1)
        for key in features3:
            for i in range(features3[key].size(0)):
                for j in range(0, features3[key].size(1), STEPS):
                    inp = torch.zeros(1, 2 * STEPS, features1[key].size(2), features1[key].size(3))
                    inp[0, 0 : 2 * STEPS : 2, :, :] = features1[key][i, j:j+STEPS, :, :].unsqueeze(0)
                    inp[0, 1 : 2 * STEPS : 2, :, :] = features2[key][i, j:j+STEPS, :, :].unsqueeze(0)
                    features3[key][i,j:j+STEPS,:,:] = self.MergeNet(inp.cuda()).unsqueeze(0)       
        
        proposals, proposal_losses = self.rpn(xs, features3, targets)
        detections, detector_losses = self.roi_heads(features3, proposals, xs.image_sizes, targets)
        detections = self.transform.postprocess(detections, xs.image_sizes, original_image_sizes)

        losses = {}
        losses.update(detector_losses)
        losses.update(proposal_losses)

        if torch.jit.is_scripting():
            if not self._has_warned:
                warnings.warn("RCNN always returns a (Losses, Detections) tuple in scripting")
                self._has_warned = True
            return (losses, detections)
        else:
            return self.eager_outputs(losses, detections)

In [0]:
NetC = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)   
for p in NetC.parameters():
    p.requires_grad = False
backbone = NetC.backbone
transform = GeneralizedRCNNTransform(min_size=400,
                                     max_size=800,
                                     image_mean = [0.485, 0.456, 0.406],
                                     image_std = [0.229, 0.224, 0.225])
rpn = NetC.rpn
roi_heads = NetC.roi_heads
roi_heads.box_predictor.cls_score = nn.Linear(in_features=1024, out_features=2, bias=True)
roi_heads.box_predictor.bbox_pred = nn.Linear(in_features=1024, out_features=8, bias=True)

NetC = NETC(backbone=backbone, rpn=rpn, roi_heads=roi_heads, transform=transform)

for name, param in NetC.named_parameters():
    if param.requires_grad:
        print(name)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth



roi_heads.box_predictor.cls_score.weight
roi_heads.box_predictor.cls_score.bias
roi_heads.box_predictor.bbox_pred.weight
roi_heads.box_predictor.bbox_pred.bias
MergeNet.0.weight
MergeNet.3.weight


In [0]:
def IOU(boxA, boxB):
    l = max(boxA[0], boxB[0])
    r = min(boxA[2], boxB[2])
    t = max(boxA[1], boxB[1])
    b = max(boxA[3], boxB[3])

    intersection = max(r-l, 0) * max(b-t, 0)
    union = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1]) + (boxB[2] - boxB[0]) * (boxB[3] - boxB[1]) - intersection
    return intersection / union

def SCORE(gts, dets):
    if len(gts) == 0:
        return 0, len(dets), 0
    if len(dets) == 0:
        return 0, 0, len(gts)
    scores = np.zeros((len(dets), len(gts)))
    mask = np.ones_like(scores)
    for i, gt in enumerate(gts):
        for j, det in enumerate(dets):
            scores[j, i] = IOU(gt, det)
    TP, FP, FN = 0, 0, 0
    t = []
    while np.max(mask*scores) > 0:
        loc = np.argmax(mask * scores)
        val = np.max(mask * scores)
        mask[loc//mask.shape[1], :] = 0
        mask[:, loc%mask.shape[1]] = 0
        if val > 0.5:
            TP += 1
    FN = len(gts) - TP
    FP = len(dets) - TP
    return TP, FP, FN


In [0]:
n_epochs = 20
batch_size = 10
def imread(path, num):
    im = Image.open(path + str(num).zfill(6) + ".jpg")
    transform = T.Compose([T.ToTensor()])
    im = transform(im)
    return im.cuda()

import torch.optim as optim
from torch.optim import lr_scheduler

params = [p for p in NetC.parameters() if p.requires_grad]

optimizer = optim.Adam(params, lr=0.001)


NetC.cuda()
min_loss = np.inf
history = {'train_loss': [],
           'val_acc': []}
best = 0
for epoch in range(n_epochs):
    train_loss = []
    NetC.train()
    for i in range(0, len(trainset), batch_size):
        print("\r Epoch %d, training [%d / %d]" % (epoch+1, i//batch_size+1, int(len(trainset)/batch_size)), end='')
        optimizer.zero_grad()
        ids = trainset[i:i+batch_size]
        sample = [imread("/content/dataset/in/", num) for num in ids]
        target = [{k: v.cuda() for k, v in targets[num].items()} for num in ids]

        back = []
        for num in ids:
            random.seed(num + epoch * 1000)
            p = random.uniform(0, 1)
            if p < 0.25:
                back.append(imread("/content/dataset/bk/", num))
            elif p < 0.5:
                random.seed(num)
                pp = (n_frames // 4)
                x = random.randint(pp * (num//pp), pp * (num//pp + 1)-1)
                back.append(imread("/content/dataset/bk/", x))
            elif p < 0.75:
                back.append(imread("/content/dataset/in/", num))
            else: 
                random.seed(num)
                pp = (n_frames // 4)
                x = random.randint(pp * (num//pp), pp * (num//pp + 1)-1)
                back.append(imread("/content/dataset/in/", x))

        loss_dict = NetC(sample, back, target)
        running_loss = sum([loss for loss in loss_dict.values()])
        running_loss.backward()
        optimizer.step()
        train_loss.append(running_loss.item())
    NetC.eval()
    TP, FP, FN = 0, 0, 0
    print("")
    for i in range(0, len(valset), batch_size):
        print("\r Epoch %d, validation [%d / %d]" % (epoch+1, i//batch_size+1, int(len(valset)/batch_size)), end='')
        ids = valset[i:i+batch_size]
        sample = [imread("/content/dataset/in/", num) for num in ids]
        back = []
        for num in ids:
            random.seed(num)
            p = random.uniform(0, 1)
            if p < 0.25:
                back.append(imread("/content/dataset/bk/", num))
            elif p < 0.5:
                random.seed(num)
                pp = (n_frames // 4)
                x = random.randint(pp * (num//pp), pp * (num//pp + 1)-1)
                back.append(imread("/content/dataset/bk/", x))
            elif p < 0.75:
                back.append(imread("/content/dataset/in/", num))
            else: 
                random.seed(num)
                pp = (n_frames // 4)
                x = random.randint(pp * (num//pp), pp * (num//pp + 1)-1)
                back.append(imread("/content/dataset/in/", x))
        target = [{k: v.numpy() for k, v in targets[num].items()} for num in ids]
        gts = []
        for t in target:
            boxes = t["boxes"]
            gt = [box for box in boxes]
            gts.append(gt)
        detections = NetC(sample, back)
        results = []
        for detection in detections:
            result = []
            scores = detection["scores"].detach().cpu()
            boxes = detection["boxes"].detach().cpu()
            for j, score in enumerate(scores):
                if score > 0.5:
                    result.append(boxes[j].numpy())
            results.append(result)

        for j, gt in enumerate(gts):
            tp, fp, fn = SCORE(gt, results[j])
            TP += tp
            FP += fp
            FN += fn
    if TP == 0 and FP == 0:
        b = 0
    else: 
        Recall = TP / (TP + FN)
        Precision = TP / (TP + FP)
        b = (2 * Precision * Recall) / (Precision + Recall)
    a = np.mean(np.array(train_loss))
    print("")
    print("Epochs: %d, train loss=%f, val acc= %f" % (epoch+1, a, b))
    if b > best:
        best = b
        print("==> best model (acc = {:0.6f}), saving model...".format(b))
        best_weights = deepcopy(NetC.state_dict())
        torch.save(best_weights, "/content/drive/My Drive/889PR/bestCrandom" + str(2*STEPS) + "to" + str(STEPS) + ".pth")
    
    history["train_loss"].append(a)
    history["val_acc"].append(b)
    

 Epoch 1, training [1 / 200]

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


 Epoch 1, training [200 / 200]
 Epoch 1, validation [50 / 50]
Epochs: 1, train loss=0.357194, val acc= 0.676142
==> best model (acc = 0.676142), saving model...
 Epoch 2, training [200 / 200]
 Epoch 2, validation [50 / 50]
Epochs: 2, train loss=0.195159, val acc= 0.803896
==> best model (acc = 0.803896), saving model...
 Epoch 3, training [200 / 200]
 Epoch 3, validation [50 / 50]
Epochs: 3, train loss=0.171615, val acc= 0.828891
==> best model (acc = 0.828891), saving model...
 Epoch 4, training [200 / 200]
 Epoch 4, validation [50 / 50]
Epochs: 4, train loss=0.159956, val acc= 0.841263
==> best model (acc = 0.841263), saving model...
 Epoch 5, training [200 / 200]
 Epoch 5, validation [50 / 50]
Epochs: 5, train loss=0.151686, val acc= 0.846026
==> best model (acc = 0.846026), saving model...
 Epoch 6, training [200 / 200]
 Epoch 6, validation [50 / 50]
Epochs: 6, train loss=0.147479, val acc= 0.846312
==> best model (acc = 0.846312), saving model...
 Epoch 7, training [200 / 200]
 Ep

In [0]:
history

{'train_loss': [0.35719351679086686,
  0.19515861615538596,
  0.17161544673144818,
  0.1599555177241564,
  0.15168562401086091,
  0.1474785041064024,
  0.14394167464226484,
  0.14121955297887326,
  0.1382161197066307,
  0.13667573098093272,
  0.13555529214441775,
  0.13433661215007306,
  0.13304641470313072,
  0.13232807476073505,
  0.13100639309734105,
  0.13037075463682413,
  0.12985925544053317,
  0.12918778263032438,
  0.12900206699967384,
  0.12803275350481272],
 'val_acc': [0.6761421319796954,
  0.8038955289951305,
  0.8288907996560618,
  0.8412631578947368,
  0.8460264900662251,
  0.8463123197363,
  0.8488042156465343,
  0.8393574297188755,
  0.836591086786552,
  0.8390397481306573,
  0.8341121495327102,
  0.8207293666026871,
  0.8397006695549428,
  0.8256880733944955,
  0.8240244835501147,
  0.8252615265401007,
  0.8183873398643556,
  0.811539902585238,
  0.8184591914569032,
  0.8062476757158795]}